In [ ]:
import numpy as np

In [6]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [76]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)> 0:
            return False
        else:
            
            return True
    
    def add_queue(self, group):
        if group.get_vip():   # If current group is a VIP group, move it forward by four groups,
            enterQueue=False
            if len(self.groups) >= 4:
                for i in range(0,4):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                        self.groups.insert(4,group)
            elif len(self.groups) > 1 and len(self.groups) < 4:
                for i in range(0,len(self.groups)):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                    self.groups.insert(1,group)
        else: 
            self.groups.insert(0,group)
            
    
    def del_queue(self): #delete last=delete first come group 
        return self.groups.pop()  

In [8]:
# this class can be regarded as "class printer" in that case.
class Table:
    def __init__(self, num, size):
        self.num = num  # No. of the table
        self.size = size  # Size of the table: for group of up to 2, 4 or 6.
        
        self.currentGroup = None # Is the table occupied or not.
        
    def busy(self):
        if self.currentGroup != None:
            return True
        else:
            return False      
    
    def startNext(self, newGroup):
        self.currentGroup = newGroup
    
    def cleanTable(self):  #when one group finish thier meal, set their table's current group to none
        self.currentGroup = None
    
    def get_num(self):
        return self.num

In [130]:
class Group:
    def __init__(self, time, size, vip, groupID):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table= None # Which table the group will be assigned to
        self.timeRequest = mod_pert_random(0, 45, 120, samples=1).astype(int)  #How long will the group spend
        self.groupID=groupID
    
    def get_groupID(self):
        return self.groupID
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip
    
    def get_time_request(self): 
        return self.timeRequest

In [10]:
def tablesSetting(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Table(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Table(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Table(i+number_tables_4+number_tables_2,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [90]:
output[0].groups

[]

In [ ]:
def generation(Duration):
    # Generate size 
    size = np.random.randint(1,7,30)
    print(size)
    
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    
    print(timestamp_list)
    
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    total_timeR = [0]
    nextGroup_endTime_2={}
    
    # Add every group into corresponding queue
    groupNumb_2=0
    groupNumb_4=0
    groupNumb_6=0
    
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter], groupNumb_2))
                counter += 1
                groupNumb_2+=1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter], groupNumb_4))
                counter += 1
                groupNumb_4+=1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter], groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
                
        
    for i in range(Duration):   
        if i in nextGroup_endTime_2.values():
            for num in nextGroup_endTime_2.keys():
                if i == nextGroup_endTime_2[num]:
                    print('Clean:', num.num,'\n')
                    table_2[num.num].currentGroup = None
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
            
        
        if (len(queue_2.groups)>0):
            current_q=[]
            for item in queue_2.groups:
                current_q.append(item.get_groupID())
            print(current_q)
            if (i >= (queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                for t in table_2:
                    if (len(queue_2.groups)>0) and (not t.busy()) and (i>=(queue_2.groups[len(queue_2.groups)-1].get_stamp())):
                        print('Head GroupID:',queue_2.groups[len(queue_2.groups)-1].get_groupID())
                        nextGroup = queue_2.del_queue()
                     
                        print('Wait time:', nextGroup.wait_time(i))
                     
                     
                     
                        t.startNext(nextGroup)
                        print('Group No.:', nextGroup.get_groupID())
                        print('VIP?:', nextGroup.get_vip())
                        print('Registration time:', nextGroup.get_stamp())
                        print('Entering into table time:',i)
                         
                         
                         
                        nextGroup.table=t.num
                        print('Table No.', t.num)
                         
                        nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                        print('\n')
                        total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)
        
    return (queue_2, queue_4, queue_6)

In [1]:
def generation(Duration):
    size = np.random.randint(1,3,90)  #1-6 people for one group, and generate x groups
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2,table_4,table_6=tablesSetting(6,4,2)  #setting tables
    
    #print(table_2,table_4,table_6)
    total_timeR = []
    nextGroup_endTime_2={}
    
    groupNumb_2=1
    groupNumb_4=1
    groupNumb_6=1
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter],groupNumb_2))
                
                print('new regislation vip+No+time: ',vip[counter], groupNumb_2,i)
                
                counter += 1
                groupNumb_2 += 1
                
                current_q=[]
                for item in queue_2.groups:
                    current_q.append(item.get_groupID())
                print(current_q,'current queue_2')
                
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter],groupNumb_4))
                counter += 1
                groupNumb_4 += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter],groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
        
        if i in nextGroup_endTime_2.values():
            print('\nkeys',nextGroup_endTime_2.keys())
            print('values',nextGroup_endTime_2.values())
            for n in list(nextGroup_endTime_2.keys()):
                if i == int(nextGroup_endTime_2[n]):
                    print('Clean:', n,'\n')
                    table_2[n-1].currentGroup = None
            #table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
                    
        
        for t in table_2:
            if (not t.busy()) and (not queue_2.isEmpty()):
                
                
                print(t.get_tableNum(),'table ID')

                nextGroup = queue_2.del_queue()
                print(nextGroup.get_stamp(),'regislation time')
                print(nextGroup.wait_time(i),'wait time')
                t.startNext(nextGroup)
                print('Group can sit now: ',nextGroup.get_groupID())
                nextGroup_endTime_2[t.get_tableNum()] = i+nextGroup.get_time_request()+2  #end time for each 2-people-table
                #2min for table cleaning, rearragement and the time to walk in blabla
                print(nextGroup_endTime_2[t.get_tableNum()],'next group end time')
                total_timeR.append(int(nextGroup.get_time_request())+2)  #def generation(Duration):
                print(total_timeR,'total request time \n')

        
    #avg_timeR = sum(total_timeR)/len(total_timeR)
        

In [ ]:
def simulation(current_time, table_type, queue_type, total_time, total_timeR, nextGroup_endTime):
    
    TableFinish(current_time, nextGroup_endTime, table_type)                
        
    for t in table_type:
        if (not t.busy()) and (not queue_type.isEmpty()):
            print('table ID:', t.get_num()) # Currently available table
            nextGroup = queue_type.del_queue()
            print('Regislation time:', nextGroup.get_stamp())
            print('Wait time:', nextGroup.wait_time(current_time))
            t.startNext(nextGroup)
            print('Group can sit now:', nextGroup.get_groupID(), '\n')
            nextGroup_endTime[t.get_num()] = current_time+nextGroup.get_time_request()+2  #end time for each 2-people-table
            #2min for table cleaning, rearragement and the time to walk in blabla
            #print(nextGroup_endTime_2[t.get_num()], 'next group end time')
            total_timeR.append(int(nextGroup.get_time_request())+2)  #def generation(Duration):
                                  #print(total_timeR,'total request time \n')
                
    # Simulation duartion is done, for groups who are not assigned
    if current_time == total_time-1:#total_time: Duration
        at_least_waittime = []
        for i in range(queue_type.queue_size()):
            if len(nextGroup_endTime) > 0:
                next_finish_time = min(nextGroup_endTime.values())
                next_finish_table = min(nextGroup_endTime, key=nextGroup_endTime.get)
                unpro_next = queue_type.del_queue()
                print('Group', unpro_next.get_groupID(), 'needs to wait', 
                      int(unpro_next.wait_time(next_finish_time)),'minutes to be assigned.')
                at_least_waittime.append(int(unpro_next.wait_time(next_finish_time)))
                nextGroup_endTime.pop(next_finish_table)
            else:
                unpro_next = queue_type.del_queue()
                print('There are still', i, 'Groups in front of Group No.',
                       unpro_next.get_groupID(), 'you need to wait at least',max(at_least_waittime),'minutes to be assign')
        #avg_timeR = sum(total_timeR)/len(total_timeR)

In [ ]:
def generation(Duration):
    size = np.random.randint(1,7,50)  #3-4 people for one group, and generate x groups
    # Generate vip situation, based on the probability of 8%
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    # Generate the registration time for each group
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2,table_4,table_6=tablesSetting(6,4,2)  #Initializing tables
    
    total_timeR_2 = []
    nextGroup_endTime_2 ={} # {No. of table: the ending time of the table}
    
    total_timeR_4 = []
    nextGroup_endTime_4 ={}
    total_timeR_6 = []
    nextGroup_endTime_6 ={}
    
    groupNumb=1  #all group have their unique ID
    #groupNumb_2=1
    #groupNumb_4=1
    #groupNumb_6=1
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter],groupNumb))
                counter += 1
                groupNumb += 1
            
                print('**\nNew Regislation vip:', vip[counter])
                print('groupNo.', groupNumb)
                print('groupPeople:',size[counter])
                print('Time:', i,'\n')
                counter += 1
                groupNumb += 1
                current_q=[]
                for item in queue_4.groups:
                    current_q.append(item.get_groupID())
                print('current queue_4', current_q)
               
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter],groupNumb))
                
                print('**\nNew Regislation vip:', vip[counter])
                print('groupNo.', groupNumb)
                print('groupPeople:',size[counter])
                print('Time:', i,'\n')
                counter += 1
                groupNumb += 1
                current_q=[]
                for item in queue_4.groups:
                    current_q.append(item.get_groupID())
                print('current queue_4', current_q)
                
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter],groupNumb))
                counter += 1
                groupNumb += 1
                
                print('**\nNew Regislation vip:', vip[counter])
                print('groupNo.', groupNumb)
                print('groupPeople:',size[counter])
                print('Time:', i,'\n')
                counter += 1
                groupNumb += 1
                current_q=[]
                for item in queue_4.groups:
                    current_q.append(item.get_groupID())
                print('current queue_4', current_q)
        
        #mark 4,6 then 2 can run        
        simulation(i, table_2, queue_2, Duration, total_timeR_2, nextGroup_endTime_2)
        simulation(i, table_4, queue_4, Duration, total_timeR_4, nextGroup_endTime_4)
        simulation(i, table_6, queue_6, Duration, total_timeR_6, nextGroup_endTime_6)

In [ ]:
def TableFinish(current_time, nextGroup_endTime, table_type):
    if current_time in nextGroup_endTime.values():
        for n in list(nextGroup_endTime.keys()):
            if current_time == int(nextGroup_endTime[n]):
                print('Clean:', n,'\n')
                # Need to add a function in class table for releasing the table
                if len(table_type)==6:
                    table_type[n-1].cleanTable()
                elif len(table_type)==4:
                    table_type[n-6-1].cleanTable()
                elif len(table_type)==2:
                    table_type[n-10-1].cleanTable()
                #table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
                #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)

In [ ]:
generation(240) 